In [1]:
import numpy as np
import matplotlib
import pandas as pd
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation, rc
from IPython.display import HTML
import pgmpy as pgm
from sklearn.decomposition import PCA
import useful

%matplotlib notebook
matplotlib.rcParams['figure.figsize'] = [16, 9]
matplotlib.rcParams['figure.dpi'] = 150

In [4]:
data = np.load('../data/chorales_matrices_12_32.npy')

In [5]:
data.shape

(12, 32, 1764)

In [ ]:
# create dataframe
